# Callbacks 101

In [1]:
def fsum(X, f=None):
    """Compute the sum of `f(x)` for each element in `X`.

    Args:
        X: A sequence of integers
        f: A function with signature `f(x)`, where `x` is an integer.
            `f(x)` should return a number.
            If `f` is not specified, it defaults to the identity function.
    """
    res = 0
    for (i, x) in enumerate(X):
        if f is not None:
            res += f(x)
        else:
            res += x
        
    return res

Someone defined a function called `fsum`...

In [2]:
help(fsum)

Help on function fsum in module __main__:

fsum(X, f=None)
    Compute the sum of `f(x)` for each element in `X`.
    
    Args:
        X: A sequence of integers
        f: A function with signature `f(x)`, where `x` is an integer.
            `f(x)` should return a number.
            If `f` is not specified, it defaults to the identity function.



In [3]:
fsum([1, 2, 3, 4])

10

> `fsum(X, f=None)`

The extra argument `f` is a function...

In [4]:
def my_callback(x):
    print(f'I am in a callback! (BTW, x={x})')
    return x

... and this function is called during the execution of `fsum`

In [5]:
res = fsum([1, 2, 3, 4], my_callback)
print(f'result = {res}')

I am in a callback! (BTW, x=1)
I am in a callback! (BTW, x=2)
I am in a callback! (BTW, x=3)
I am in a callback! (BTW, x=4)
result = 10


### Fancier examples

What if we want to sum only _odd_ or _even_ integers?

In [6]:
X_all  = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Explicitly keep only odd / even integers
X_odd  = [1, 3, 5, 7, 9]
X_even = [2, 4, 6, 8, 10]  

In [7]:
print("sum(all)  :", fsum(X_all))
print("sum(odd)  :", fsum(X_odd))
print("sum(even) :", fsum(X_even))

sum(all)  : 55
sum(odd)  : 25
sum(even) : 30


Could we do that without creating new lists?

Yes! Just define a callback that skips odd/even integers

In [8]:
def even_only(x): 
    if (x % 2 == 1):
        print(f'Callback #1: skipping odd element x={x}.')
        return 0
    
    return x
    
def odd_only(x):
    if (x % 2 == 0):
        print(f'Callback #1: skipping even element x={x}.')
        return 0
    
    return x

In [9]:
res = fsum(X_all, even_only)
print(f'sum(even) : {res}')

Callback #1: skipping odd element x=1.
Callback #1: skipping odd element x=3.
Callback #1: skipping odd element x=5.
Callback #1: skipping odd element x=7.
Callback #1: skipping odd element x=9.
sum(even) : 30


In [10]:
res = fsum(X_all, odd_only)
print(f'sum(odd)  : {res}')

Callback #1: skipping even element x=2.
Callback #1: skipping even element x=4.
Callback #1: skipping even element x=6.
Callback #1: skipping even element x=8.
Callback #1: skipping even element x=10.
sum(odd)  : 25


# Closures 101

MIP callbacks in Julia and python use so-called _closures_.

 > A closure is a record storing a function together with an environment

In English: you can define a function that captures a variable defined _outside_ the function.

### `lambda` functions are closures!

In [11]:
a = 1
g = lambda x: (a+x)

In [12]:
g(1)

2

In [13]:
a = 3
g(1)

4

In [14]:
a = "hello"
g(1)

TypeError: can only concatenate str (not "int") to str

### Closures within a function

In [15]:
def modulo_sum(X):
    # `n` and `U` are defined within `modulo_sum`
    n = 0
    U = []
    
    # The definition of `f` "captures" the variables `n` and `U`
    # This allows us to modify `U` within the callback
    def f(x):
        U.append(x)
        return x ** (n)
    
    for k in range(1, 4):
        n = k             # <-- this implicitly changes f!
        res = fsum(X, f)  # <-- will modify U
        
        print(f'Σ(x^{k}) = {res}')
    
    return U

In [16]:
X = range(1, 4)
U = modulo_sum(X)
print('U = ', U)

Σ(x^1) = 6
Σ(x^2) = 14
Σ(x^3) = 36
U =  [1, 2, 3, 1, 2, 3, 1, 2, 3]


## When is this useful / needed?

* To monitor a lower / upper bound within the optimization
* To keep track of all feasible solution (or just the best)
* To access user-defined data structures
  --> e.g., a Benders' subproblem